# If this isn't sinister, I don't know what is </font>

---  

This notebook contains some analysis on data available through <a href='https://joshuaproject.net/'>The Joshua Project </a>- which is described<sup>[1]</sup> as an organization seeking to highlight the ethnic groups of the world with the least followers of evangelical Christianity.

### Importing the tools and libraries

In [67]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import gmaps
import math
import gmplot
import iso3166 as iso
from iso3166 import countries
import geocoder

%matplotlib inline
rcParams['figure.figsize'] = (17,17)

### Importing the dataset  
  
The dataset is available for download [here]().

In [2]:
data = pd.read_csv('data/AllPeoplesByCountry.csv', skiprows=1, skipfooter=16, engine='python')

In [3]:
data.head(2)

,ROG3,Ctry,PeopleID3,ROP3,PeopNameAcrossCountries,PeopNameInCountry,Population,JPScale,LeastReached,ROL3,...,RegionCode,RegionName,ROG2,Continent,10_40Window,RaceCode,Latitude,Longitude,WorkersMin,WorkersMax
0,AF,Afghanistan,14372,107989.0,Afghan,Afghan,8207000.0,1,Y,prs,...,5,Central Asia,ASI,Asia,Y,CNT24f,31.15621,62.14612,165,170
1,AF,Afghanistan,19409,100096.0,Afshari,Afshari,13000.0,1,Y,azb,...,5,Central Asia,ASI,Asia,Y,MSY41a,34.44796,69.28976,1,2


### Quick plot of world distribution

In [53]:
world_map = gmplot.GoogleMapPlotter('20.5937', '78.9629',12)

In [54]:
for i in range(1,35):
    world_map.heatmap(data['Latitude'][500*(i-1):500*i],data['Longitude'][500*(i-1):500*i])

In [55]:
world_map.draw('World_Map.html')

### Distribution by countries

In [4]:
import plotly.plotly as py
import plotly

In [5]:
PLOTLY_API_KEY = 'UtfalzPfTgzxCcaORFXn'

In [6]:
plotly.tools.set_credentials_file(username='JanakAJain', api_key=PLOTLY_API_KEY)

In [7]:
wdata = pd.DataFrame(data.groupby(['Ctry']).count()['ROG3'].sort_values(ascending=False))

In [8]:
wdata = wdata.reset_index()

In [9]:
wdata.head(2)

,Ctry,ROG3
0,India,2510
1,Papua New Guinea,885


In [10]:
wdata['GCode'] = ''
for i, row in wdata.iterrows():
    try:
        wdata.loc[i,'GCode'] = countries.get(wdata.loc[i,'Ctry'])[2]
    except:
        continue

In [11]:
#### Note: Some 

In [12]:
wdata = pd.read_csv('geo_data_codes.csv')

In [13]:
wdata.head()

,Unnamed: 0,Ctry,ROG3,GCode
0,0,India,2510,IND
1,1,Papua New Guinea,885,PNG
2,2,Indonesia,779,IDN
3,3,Nigeria,544,NGA
4,4,China,544,CHN


In [14]:
def plot_on_world_map(data, z='ROG3',title='Number of target groups'):
    
    data = [ dict(
        type = 'choropleth',
        locations = wdata['GCode'],
        z = wdata[z],
        text = wdata['Ctry'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = title),
      ) ]
    
    layout = dict(
    title = 'Joshua Project Target Areas - '+ z + '<br>Source:\
            <a href="https://joshuaproject.net/resources/datasets/1">\
            Data Available Here</a>',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
            )
        )
    )
    
    fig = dict( data=data, layout=layout )
    
    return py.iplot( fig, validate=False, filename='d3-world-map' )

In [15]:
plot_on_world_map(wdata,'ROG3')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~JanakAJain/0 or inside your plot.ly account where it is named 'd3-world-map'


In [16]:
wdata = wdata.join(pd.DataFrame(data.groupby(['Ctry']).sum()['Population'].sort_values(ascending=False)), on='Ctry', how = 'left')

In [17]:
plot_on_world_map(wdata,'Population','Population')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~JanakAJain/0 or inside your plot.ly account where it is named 'd3-world-map'


### Let's look at India

In [18]:
data.columns.values

array(['ROG3', 'Ctry', 'PeopleID3', 'ROP3', 'PeopNameAcrossCountries',
       'PeopNameInCountry', 'Population', 'JPScale', 'LeastReached',
       'ROL3', 'PrimaryLanguageName', 'BibleStatus', 'RLG3',
       'PrimaryReligion', 'PercentAdherents', 'PercentEvangelical',
       'PeopleID1', 'ROP1', 'AffinityBloc', 'PeopleID2', 'ROP2',
       'PeopleCluster', 'CountOfCountries', 'RegionCode', 'RegionName',
       'ROG2', 'Continent', '10_40Window', 'RaceCode', 'Latitude',
       'Longitude', 'WorkersMin', 'WorkersMax'], dtype=object)

In [19]:
india = data[data['Ctry'] == 'India']

In [20]:
india.groupby(['PrimaryReligion']).count()['ROG3'].sort_values(ascending = False)

PrimaryReligion
Hinduism            1800
Islam                380
Christianity         135
Other / Small         93
Buddhism              56
Unknown               45
Ethnic Religions       1
Name: ROG3, dtype: int64

In [60]:
india_top_christian_targets = india[(india['PrimaryReligion'] == 'Christianity') & (india['JPScale'] >= 4)]

In [61]:
india_top_c_lat = india_top__christian_targets['Latitude'].astype(float)
india_top_c_lon = india_top__christian_targets['Longitude'].astype(float)

In [62]:
india_top_c_map = gmplot.GoogleMapPlotter('26.1445', '91.7362','5')

In [63]:
india_top_c_map.heatmap(india_top_c_lat, india_top_c_lon)

In [64]:
india_top_c_map.draw("India_Top_Christian_Targets.html")

In [65]:
india_top_christian_targets[['Latitude','Longitude']]

,Latitude,Longitude
6032,25.674381,90.331111
6902,23.094450,93.188852
6904,22.980801,93.048139
6905,22.368490,93.000000
6906,23.845230,92.879000
6907,23.845000,92.870000


In [85]:
s = geocoder.google([23.094450,93.188852], method='reverse')

In [86]:
s.json

{'accuracy': 'GEOMETRIC_CENTER',
 'address': 'Unnamed Road, Khuangthing, Mizoram 796320, India',
 'bbox': {'northeast': [23.1011408, 93.2001945],
  'southwest': [23.0814655, 93.1960299]},
 'city': 'Khuangthing',
 'confidence': 7,
 'country': 'IN',
 'county': 'Champhai',
 'encoding': 'utf-8',
 'lat': 23.0915817,
 'lng': 93.19720730000002,
 'location': '23.09445, 93.188852',
 'ok': True,
 'place': 'ChIJvRgHvELkTDcRLLbAjiYaKNg',
 'postal': '796320',
 'provider': 'google',
 'quality': 'route',
 'state': 'MZ',
 'status': 'OK',
 'status_code': 200,
 'street': 'Unnamed Road'}

#### All these vulnerable groups belong to (perhaps the most distant) states in the North East.

### References

1. https://en.wikipedia.org/wiki/Joshua_Project

In [ ]:
data.columns.values